In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import numpy as np



userdf = pd.read_pickle("/content/drive/My Drive/Test 500x500 /userdf.pkl")
matrix = pd.read_pickle("/content/drive/My Drive/Test 500x500 /500x500matrix.pkl")
rentlist = pd.read_pickle("/content/drive/My Drive/Test 500x500 /500x500rentlist.pkl")

In [18]:
matrix1 = matrix.T

matrix1.head()

회원번호,126.0,445.0,489.0,687.0,829.0,2180.0,2646.0,2942.0,3387.0,4043.0,4265.0,4724.0,5169.0,5294.0,5477.0,6221.0,6389.0,7003.0,7441.0,7514.0,7564.0,7631.0,7652.0,7655.0,7823.0,8299.0,8572.0,8631.0,8693.0,8929.0,9156.0,9961.0,10404.0,10859.0,11222.0,11538.0,11831.0,12085.0,12209.0,13488.0,...,127238.0,127256.0,127315.0,127552.0,128562.0,128615.0,129037.0,130429.0,130456.0,130488.0,130626.0,130645.0,130672.0,130983.0,131104.0,131667.0,131711.0,131834.0,131845.0,132364.0,132701.0,132921.0,133008.0,133273.0,133369.0,133725.0,134033.0,134421.0,134432.0,134818.0,134843.0,135175.0,135197.0,135534.0,135691.0,135780.0,136179.0,136271.0,136638.0,136640.0
ISBN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9780060741693,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9780198480648,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9780439113212,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9780679864417,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9780679884491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
import numpy as np
from tqdm import tqdm
# 유저의 연관성을 계산하는 식 거리 계산은 상관계수 방법인 피어슨(pearson) 방법으로 계산함.
# 피어슨 함수(def) 선언
from numpy import dot
from numpy.linalg import norm

def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))  
  
# 유사도 측정
def recommend(input_item, matrix, n):   
    
    # 결과값 리스트선언
    result = []
    
    # 모든 user들의 유사도를 구하기 위해 한명씩 뽑음.
    for item in matrix.index:
        
        # 같은 이름의 책은 연관성 비교에서 제외
        if item == input_item:
            result.append(1.00)
            continue

        # rating comparison
        # 유저의 연관성을 계산
        cor = cos_sim(matrix.loc[input_item], matrix.loc[item])        
        
        if np.isnan(cor):
            continue
        # cor 점수가 NAN 값이 아니면 result 리스트에 ISBN,cor,도서명,주제분류번호를 저장    
        else:
            result.append('{:.4f}'.format(cor))    
    # 후보 상위 n개만큼 리턴!
    return result[:n]

  
# # 결과물인 유저 유사도 매트릭스의 틀을 선언  
# userdf = pd.DataFrame(0,index = matrix.index ,columns = matrix.index)
# # 유저 x 유저의 유사도 매트릭스에 채우기 위한 반복문  
# for i in tqdm(matrix.index, desc = "총 진행률"):  
# recommend_result = recommend(126, matrix, 500)
# len(recommend_result)
#   userdf[i] = recommend_result

# userdf


In [60]:
itemdf = pd.DataFrame(0,index = matrix1.index ,columns = matrix1.index)
for i in tqdm(matrix1.index):  
  recommend_result = recommend(i, matrix1, 500)
  itemdf[i] = recommend_result


100%|██████████| 500/500 [02:03<00:00,  4.13it/s]


In [0]:
pd.to_pickle(itemdf, "/content/drive/My Drive/Test 500x500 /itemdf.pkl")

In [62]:
itemdf

ISBN,9780060741693,9780198480648,9780439113212,9780679864417,9780679884491,9780743461511,9780746054116,9780746054925,9781405052306,9781409505488,9781409520696,9781783442904,9788901028194,9788901033518,9788901052922,9788901054926,9788901072531,9788901101330,9788901114941,9788901114965,9788901121390,9788901162294,9788911021406,9788911027101,9788911030088,9788914017420,9788915073098,9788915084858,9788917198799,9788925548968,9788925550916,9788925551531,9788925554167,9788925554761,9788926387061,9788926387153,9788926390214,9788926391051,9788926391068,9788926391235,...,9788995151204,9788995501443,9788995688991,9788996134619,9788996134664,9788996145622,9788996640479,9788996746416,9788996842262,9788996881421,9788996991342,9788997089345,9788997174485,9788998515065,9791155427095,9791156390480,9791156394747,9791158461102,9791158461409,9791158510619,9791158680657,9791160486735,9791161231686,9791170260752,9791185205403,9791185343457,9791185343693,9791185431550,9791185431611,9791185431864,9791186340691,9791186472767,9791186560112,9791186560167,9791186602096,9791187481324,9791195049981,9791195568758,9791195788583,9788993632583
ISBN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9780060741693,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2887,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9780198480648,0.0000,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2887,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9780439113212,0.0000,0.0000,1,0.0000,0.0000,0.0000,0.2357,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2357,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2041,0.0000,0.2041,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2041,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9780679864417,0.0000,0.0000,0.0000,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
9780679884491,0.0000,0.0000,0.0000,0.0000,1,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.2887,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.00

In [0]:
# 500x500을 뽑을 때 유저를 기반으로 추출했기 때문에 아이템 유사도 점수가 상대적으로 많이 낮음...
# 다시뽑아야 되나 귀찮네..
# 일단 해봄..

In [0]:
# 출처 : https://proinlab.com/archives/2103
# 잘못알았다!!!!!! 아이템기반 협업필터링은 유사도를 구하고 그 점수를 이용해 책을 추천해주는 방식이다!!...
# 즉 유사도 --> 예측값 반영한 추천 리스트 순서가 아니라 순수히 유사도만 가지고 리스트를 뽑은게 아이템기반 협업필터링.. 그래서 속도가 빨랐구나...


In [88]:
itemdf.loc[9780060741693]

ISBN
9780060741693          1
9780198480648     0.0000
9780439113212     0.0000
9780679864417     0.0000
9780679884491     0.0000
                   ...  
9791187481324     0.0000
9791195049981     0.0000
9791195568758     0.0000
9791195788583     0.0000
9788993632583     0.0000
Name: 9780060741693, Length: 500, dtype: object

In [0]:
a = itemdf[itemdf[9780060741693]!= '0.0000'][9780060741693]


In [153]:
a = pd.to_numeric(a)
a = a.sort_values(ascending=False)
a[1:11].index

Index([9788964355190, 9788950936136, 9788950936105, 9788950921736,
       9788930202558, 9788928302802, 9788953315464, 9788937845390,
       9788930202589, 9788925554761],
      dtype='object', name='ISBN')

In [0]:
for a in itemdf.index:
  print(a)

In [0]:
from tqdm import tqdm
prelist = {}
def getitemToplist(item,itemdf):
  result = itemdf[itemdf[item]!= '0.0000'][item]
  result = pd.to_numeric(result)
  result = result.sort_values(ascending=False)
  return result[1:11].index.values

In [174]:
for a in tqdm(itemdf.index, desc = "진행상황"):
  prelist[str(a)] = getitemToplist(a,itemdf)

진행상황: 100%|██████████| 500/500 [00:00<00:00, 556.00it/s]


In [0]:
itemprelist = prelist

In [0]:
pd.to_pickle(itemprelist, "/content/drive/My Drive/Test 500x500 /itemtopNpredict.pkl")

회원번호,126.0,445.0,489.0,687.0,829.0,2180.0,2646.0,2942.0,3387.0,4043.0,4265.0,4724.0,5169.0,5294.0,5477.0,6221.0,6389.0,7003.0,7441.0,7514.0,7564.0,7631.0,7652.0,7655.0,7823.0,8299.0,8572.0,8631.0,8693.0,8929.0,9156.0,9961.0,10404.0,10859.0,11222.0,11538.0,11831.0,12085.0,12209.0,13488.0,...,127238.0,127256.0,127315.0,127552.0,128562.0,128615.0,129037.0,130429.0,130456.0,130488.0,130626.0,130645.0,130672.0,130983.0,131104.0,131667.0,131711.0,131834.0,131845.0,132364.0,132701.0,132921.0,133008.0,133273.0,133369.0,133725.0,134033.0,134421.0,134432.0,134818.0,134843.0,135175.0,135197.0,135534.0,135691.0,135780.0,136179.0,136271.0,136638.0,136640.0
ISBN,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9780060741693,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9780198480648,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9780439113212,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9780679864417,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9780679884491,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
